<a href="https://colab.research.google.com/github/sahilfaizal01/Health_Analysis_Engine/blob/main/GPT3_5_Fine_Tuning_Medical_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing necessary libraries**

In [1]:
!pip install datasets tiktoken openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successf

# **Importing libraries**

In [56]:
import os
import openai
from openai import OpenAI


openai.api_key = ''

In [3]:
import pandas as pd
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [59]:
import json
from pathlib import Path

# **Importing the data**

In [4]:
!git clone https://huggingface.co/datasets/gamino/wiki_medical_terms

Cloning into 'wiki_medical_terms'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16 (from 1)
Unpacking objects: 100% (16/16), 2.70 KiB | 276.00 KiB/s, done.


# Data Insights:-
## User - Patient (Input)
## Assistant - Doctor (Output)

# **Preparing the data**

In [23]:
data_path = "/content/wiki_medical_terms/wiki_medical_terms.parquet"

In [24]:
df = pd.read_parquet(data_path, engine="pyarrow")
df.columns = ['User','Assistant']
df.head()

,User,Assistant
0,Paracetamol poisoning,"Paracetamol poisoning, also known as acetamino..."
1,Acromegaly,Acromegaly is a disorder that results from exc...
2,Actinic keratosis,"Actinic keratosis (AK), sometimes called solar..."
3,Congenital adrenal hyperplasia,Congenital adrenal hyperplasia (CAH) is a grou...
4,Adrenocortical carcinoma,Adrenocortical carcinoma (ACC) is an aggressi...


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6861 entries, 0 to 7275
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       6861 non-null   object
 1   Assistant  6861 non-null   object
dtypes: object(2)
memory usage: 160.8+ KB


In [26]:
df.isnull().sum()

,0
User,0
Assistant,0


In [27]:
def prep_openai_format(conversation, system_message=None):
  messages = []
  messages.append({
      "role": "system",
      "content": system_message
    })
  messages.append({
      "role": "user",
      "content": conversation['User']
    })
  messages.append({
      "role": "assistant",
      "content": conversation['Assistant']
    })
  output_dict = {
        "messages": messages
    }

  return output_dict

In [28]:
system_message = "You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor."

In [29]:
dataset = []
for i in range(len(df)):
  record = prep_openai_format(df.iloc[i], system_message=system_message)
  dataset.append(record)

In [30]:
# initial dataset
print("Examples : ", len(dataset))
print("First Example:")
for message in dataset[0]['messages']:
  print(message)

Examples :  6861
First Example:
{'role': 'system', 'content': 'You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor.'}
{'role': 'user', 'content': 'Paracetamol poisoning'}
{'role': 'assistant', 'content': 'Paracetamol poisoning, also known as acetaminophen poisoning, is caused by excessive use of the medication paracetamol (acetaminophen). Most people have few or non-specific symptoms in the first 24 hours following overdose. These include feeling tired, abdominal pain, or nausea. This is typically followed by a couple of days without any symptoms, after which yellowish skin, blood clotting problems, and confusion occurs as a result of liver failure. Additional complications may include kidney failure, pancreatitis, low blood sugar, and lactic acidosis. If death does not occur, people tend to recover fully over a couple of weeks. Without treatment, death from toxicity occurs 4 to 18 days later.Paracetamol poisoning can occur accidental

In [31]:
for message in dataset[1]['messages']:
  print(message)

{'role': 'system', 'content': 'You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor.'}
{'role': 'user', 'content': 'Acromegaly'}
{'role': 'assistant', 'content': 'Acromegaly is a disorder that results from excess growth hormone (GH) after the growth plates have closed. The initial symptom is typically enlargement of the hands and feet. There may also be an enlargement of the forehead, jaw, and nose. Other symptoms may include joint pain, thicker skin, deepening of the voice, headaches, and problems with vision. Complications of the disease may include type 2 diabetes, sleep apnea, and high blood pressure.Acromegaly is usually caused by the pituitary gland producing excess growth hormone. In more than 95% of cases the excess production is due to a benign tumor, known as a pituitary adenoma. The condition is not inherited from a persons parents. Acromegaly is rarely due to a tumor in another part of the body. Diagnosis is by measuring gr

In [32]:
for message in dataset[6860]['messages']:
  print(message)

{'role': 'system', 'content': 'You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor.'}
{'role': 'user', 'content': 'Occipital neuralgia'}
{'role': 'assistant', 'content': 'Occipital neuralgia (ON) is a painful condition affecting the posterior head in the distributions of the greater occipital nerve (GON), lesser occipital nerve (LON), third occipital nerve (TON), or a combination of the three. It is paroxysmal, lasting from seconds to minutes, and often consists of lancinating pain that directly results from the pathology of one of these nerves. It is paramount that physicians understand the differential diagnosis for this condition and specific diagnostic criteria. There are multiple treatment modalities, several of which have well-established efficacy in treating this condition.\n\nSigns and symptoms\nPatients presenting with a headache originating at the posterior skull base should be evaluated for ON. This condition typically pres

# **Error Checking**

In [33]:
def error_checker(dataset):
  format_errors = defaultdict(lambda: defaultdict(int))

  for idx, ex in enumerate(dataset):
      if not isinstance(ex, dict):
          format_errors["data_type"][idx] += 1
          continue

      messages = ex.get("messages", None)
      if not messages:
          format_errors["missing_messages_list"][idx] += 1
          continue

      for message_idx, message in enumerate(messages):
          if "role" not in message or "content" not in message:
              format_errors["message_missing_key"][idx] += 1

          if any(k not in ("role", "content", "name") for k in message):
              format_errors["message_unrecognized_key"][idx] += 1

          if message.get("role", None) not in ("system", "user", "assistant"):
              format_errors["unrecognized_role"][idx] += 1

          content = message.get("content", None)
          if not content or not isinstance(content, str):
              format_errors["missing_content"][idx] += 1

      if not any(message.get("role", None) == "assistant" for message in messages):
          format_errors["example_missing_assistant_message"][idx] += 1

  # Print errors with indices
  if format_errors:
      print("Found errors:")
      for error_type, errors in format_errors.items():
          for index, count in errors.items():
              print(f"Error type '{error_type}' at index {index}: {count} occurrences")
  else:
      print("No errors found")

error_checker(dataset)

Found errors:
Error type 'missing_content' at index 6419: 1 occurrences


In [34]:
for message in dataset[6419]['messages']:
  print(message)

{'role': 'system', 'content': 'You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor.'}
{'role': 'user', 'content': 'Involvement'}
{'role': 'assistant', 'content': ''}


In [35]:
del dataset[6419]

In [36]:
len(dataset)

6860

In [37]:
error_checker(dataset)

No errors found


# **Token Counter**

In [38]:
encoding = tiktoken.get_encoding("cl100k_base")

In [42]:
# helper functions to token counting
def from_message_num_tokens(messages, tokens_per_message=3, tokens_per_name=1):
  num_tokens = 0
  for message in messages:
    num_tokens += tokens_per_message
    for key, value in message.items():
      num_tokens += len(encoding.encode(value))
      if key=="name":
        num_tokens += tokens_per_name

  num_tokens +=3
  return num_tokens

def from_message_num_assistant_tokens(messages):
  num_tokens = 0
  for message in messages:
    if message["role"] == "assistant":
      num_tokens +=len(encoding.encode(message["content"]))

  return num_tokens

def print_overview(values, name):
  print(f"\n #### Distribution of {name}:")
  print(f"min / max: {min(values)}, {max(values)}")
  print(f"mean / median: {np.mean(values)}, {np.median(values)}")
  print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [43]:
# tokens counts and warnings - from OpenAI cookbook

def tokens_counts_warnings(dataset):
  n_missing_system = 0
  n_missing_user = 0
  n_messages = []
  convo_lens = []
  assistant_message_lens = []

  for ex in dataset:
      messages = ex["messages"]
      if not any(message["role"] == "system" for message in messages):
          n_missing_system += 1
      if not any(message["role"] == "user" for message in messages):
          n_missing_user += 1
      n_messages.append(len(messages))
      convo_lens.append(from_message_num_tokens(messages))
      assistant_message_lens.append(from_message_num_assistant_tokens(messages))

  print("Num examples missing system message:", n_missing_system)
  print("Num examples missing user message:", n_missing_user)

  print_overview(n_messages, "num_messages_per_example")
  print_overview(convo_lens, "num_total_tokens_per_example")

  print_overview(assistant_message_lens, "num_assistant_tokens_per_example")

  n_too_long = sum(l > 4096 for l in convo_lens)
  print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

  return convo_lens

In [46]:
# Pricing and default n_epochs estimate

def cost_checker(dataset):
  MAX_TOKENS_PER_EXAMPLE = 4096

  convo_lens = tokens_counts_warnings(dataset)

  TARGET_EPOCHS = 3
  MIN_TARGET_EXAMPLES = 100
  MAX_TARGET_EXAMPLES = 25000
  MIN_DEFAULT_EPOCHS = 1
  MAX_DEFAULT_EPOCHS = 25

  n_epochs = TARGET_EPOCHS
  n_train_examples = len(dataset)
  if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
      n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
  elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
      n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

  n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
  chargeable_tokens = n_epochs * n_billing_tokens_in_dataset
  print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
  print(f"By default, you'll train for {n_epochs} epochs on this dataset")
  print(f"By default, you'll be charged for ~{chargeable_tokens} tokens")
  token_per_million = (chargeable_tokens/(1000000))
  print("Total cost in USD:",3 * token_per_million)

In [47]:
cost_checker(dataset)# ie; needs 32 million tokens

Num examples missing system message: 0
Num examples missing user message: 0

 #### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

 #### Distribution of num_total_tokens_per_example:
min / max: 52, 35032
mean / median: 1938.0801749271136, 1158.0
p5 / p95: 200.9000000000001, 4583.900000000003

 #### Distribution of num_assistant_tokens_per_example:
min / max: 15, 34993
mean / median: 1897.9734693877551, 1119.0
p5 / p95: 160.9000000000001, 4543.1

815 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~10973237 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~32919711 tokens
Total cost in USD: 98.75913299999999


# **Reducing the data requirement**

In [48]:
cost_checker(dataset[:1000])

Num examples missing system message: 0
Num examples missing user message: 0

 #### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

 #### Distribution of num_total_tokens_per_example:
min / max: 63, 35032
mean / median: 2835.33, 2053.0
p5 / p95: 396.70000000000005, 6199.200000000001

 #### Distribution of num_assistant_tokens_per_example:
min / max: 22, 34993
mean / median: 2795.529, 2012.0
p5 / p95: 357.70000000000005, 6162.1

206 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~2170509 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~6511527 tokens
Total cost in USD: 19.534581


In [49]:
cost_checker(dataset[:500])

Num examples missing system message: 0
Num examples missing user message: 0

 #### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

 #### Distribution of num_total_tokens_per_example:
min / max: 86, 19366
mean / median: 3217.604, 2457.0
p5 / p95: 504.0, 6934.600000000001

 #### Distribution of num_assistant_tokens_per_example:
min / max: 47, 19328
mean / median: 3178.006, 2414.0
p5 / p95: 464.9, 6896.500000000001

130 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~1196684 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~3590052 tokens
Total cost in USD: 10.770156


In [50]:
cost_checker(dataset[:250])

Num examples missing system message: 0
Num examples missing user message: 0

 #### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

 #### Distribution of num_total_tokens_per_example:
min / max: 102, 19366
mean / median: 3524.932, 2593.5
p5 / p95: 566.5, 7241.999999999998

 #### Distribution of num_assistant_tokens_per_example:
min / max: 60, 19328
mean / median: 3485.328, 2555.0
p5 / p95: 524.7, 7202.199999999999

75 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~633318 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~1899954 tokens
Total cost in USD: 5.6998619999999995


In [51]:
cost_checker(dataset[:125])

Num examples missing system message: 0
Num examples missing user message: 0

 #### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

 #### Distribution of num_total_tokens_per_example:
min / max: 103, 19366
mean / median: 3544.088, 2883.0
p5 / p95: 512.0, 7126.0

 #### Distribution of num_assistant_tokens_per_example:
min / max: 64, 19328
mean / median: 3504.592, 2843.0
p5 / p95: 475.0, 7084.0

38 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~326365 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~979095 tokens
Total cost in USD: 2.937285


In [52]:
new_dataset = dataset[:125]

# **Saving the dataset**

In [54]:
def save_to_jsonl(conversations, file_path):
  with open(file_path, 'w') as file:
    for conversation in conversations:
      json_line = json.dumps(conversation)
      file.write(json_line + '\n')

In [55]:
# train dataset

save_to_jsonl(dataset[:100], '/content/wiki_medical_term_task_train.jsonl')

# validation dataset

save_to_jsonl(dataset[100:125], '/content/wiki_medical_term_task_validation.jsonl')

In [57]:
client = OpenAI(
    api_key= openai.api_key,
)

In [58]:
training_dataset_file_name = '/content/wiki_medical_term_task_train.jsonl'
validation_dataset_file_name = '/content/wiki_medical_term_task_validation.jsonl'

In [60]:
training_response = client.files.create(
    file=Path(training_dataset_file_name),
    purpose="fine-tune"
)

In [61]:
training_response

FileObject(id='file-5c4JoGvxeo9lDJ3ryzyMyNsz', bytes=1839624, created_at=1724247452, filename='wiki_medical_term_task_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [62]:
training_file_id = training_response.id
training_file_id

'file-5c4JoGvxeo9lDJ3ryzyMyNsz'

In [63]:
validation_response = client.files.create(
    file=Path(validation_dataset_file_name),
    purpose="fine-tune"
)

In [64]:
validation_response

FileObject(id='file-TC6MuUUKm4zZVfvMhgMA8dbq', bytes=292850, created_at=1724247482, filename='wiki_medical_term_task_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [65]:
validation_file_id = validation_response.id
validation_file_id

'file-TC6MuUUKm4zZVfvMhgMA8dbq'

# **Starting fine-tuning**

In [67]:
response = client.fine_tuning.jobs.create(
    model = "gpt-3.5-turbo",
    training_file = training_file_id,
    validation_file = validation_file_id,
    suffix="wiki-medical-terms"
)

In [68]:
response

FineTuningJob(id='ftjob-pp0YXxTT9l9yTK55VrIUL4aD', created_at=1724247641, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-OAr1jvg8os1Ah0dcvn3JHKaZ', result_files=[], seed=2097429054, status='validating_files', trained_tokens=None, training_file='file-5c4JoGvxeo9lDJ3ryzyMyNsz', validation_file='file-TC6MuUUKm4zZVfvMhgMA8dbq', estimated_finish=None, integrations=[], user_provided_suffix='wiki-medical-terms')

In [69]:
job_id = response.id
job_id

'ftjob-pp0YXxTT9l9yTK55VrIUL4aD'

In [70]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-pp0YXxTT9l9yTK55VrIUL4aD', created_at=1724247641, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-OAr1jvg8os1Ah0dcvn3JHKaZ', result_files=[], seed=2097429054, status='queued', trained_tokens=None, training_file='file-5c4JoGvxeo9lDJ3ryzyMyNsz', validation_file='file-TC6MuUUKm4zZVfvMhgMA8dbq', estimated_finish=None, integrations=[], user_provided_suffix='wiki-medical-terms'), FineTuningJob(id='ftjob-gleohJsNy3CHvZBj87z1UP91', created_at=1724247572, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-OAr1jvg8os1Ah0dcvn3JHKaZ', resu

In [71]:
client.fine_tuning.jobs.retrieve('ftjob-pp0YXxTT9l9yTK55VrIUL4aD')

FineTuningJob(id='ftjob-pp0YXxTT9l9yTK55VrIUL4aD', created_at=1724247641, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-OAr1jvg8os1Ah0dcvn3JHKaZ', result_files=[], seed=2097429054, status='running', trained_tokens=None, training_file='file-5c4JoGvxeo9lDJ3ryzyMyNsz', validation_file='file-TC6MuUUKm4zZVfvMhgMA8dbq', estimated_finish=None, integrations=[], user_provided_suffix='wiki-medical-terms')

In [80]:
job_response = client.fine_tuning.jobs.list_events(fine_tuning_job_id='ftjob-pp0YXxTT9l9yTK55VrIUL4aD')

In [81]:
job_response

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-xPSCbwOD8LWeo2hc11GNZ0RJ', created_at=1724248839, level='info', message='Step 256/300: training loss=1.02', object='fine_tuning.job.event', data={'step': 256, 'train_loss': 1.0174081325531006, 'total_steps': 300, 'train_mean_token_accuracy': 0.7306666374206543}, type='metrics'), FineTuningJobEvent(id='ftevent-ntfKYR3zNUyrisFeqSvnwMjK', created_at=1724248834, level='info', message='Step 255/300: training loss=0.64', object='fine_tuning.job.event', data={'step': 255, 'train_loss': 0.6383346319198608, 'total_steps': 300, 'train_mean_token_accuracy': 0.8427051901817322}, type='metrics'), FineTuningJobEvent(id='ftevent-evaWUYjZDLWbNDFveZ0u5Ru3', created_at=1724248832, level='info', message='Step 254/300: training loss=0.84', object='fine_tuning.job.event', data={'step': 254, 'train_loss': 0.8424683809280396, 'total_steps': 300, 'train_mean_token_accuracy': 0.7723120450973511}, type='metrics'), FineTuningJobEvent(id='fte

In [82]:
events = job_response.data
events

[FineTuningJobEvent(id='ftevent-xPSCbwOD8LWeo2hc11GNZ0RJ', created_at=1724248839, level='info', message='Step 256/300: training loss=1.02', object='fine_tuning.job.event', data={'step': 256, 'train_loss': 1.0174081325531006, 'total_steps': 300, 'train_mean_token_accuracy': 0.7306666374206543}, type='metrics'),
 FineTuningJobEvent(id='ftevent-ntfKYR3zNUyrisFeqSvnwMjK', created_at=1724248834, level='info', message='Step 255/300: training loss=0.64', object='fine_tuning.job.event', data={'step': 255, 'train_loss': 0.6383346319198608, 'total_steps': 300, 'train_mean_token_accuracy': 0.8427051901817322}, type='metrics'),
 FineTuningJobEvent(id='ftevent-evaWUYjZDLWbNDFveZ0u5Ru3', created_at=1724248832, level='info', message='Step 254/300: training loss=0.84', object='fine_tuning.job.event', data={'step': 254, 'train_loss': 0.8424683809280396, 'total_steps': 300, 'train_mean_token_accuracy': 0.7723120450973511}, type='metrics'),
 FineTuningJobEvent(id='ftevent-uZ6ApAUWkqiOnzzKiR3YbGsA', creat

In [83]:
for event in events:
  print(event.message)

Step 256/300: training loss=1.02
Step 255/300: training loss=0.64
Step 254/300: training loss=0.84
Step 253/300: training loss=1.30
Step 252/300: training loss=0.20
Step 251/300: training loss=0.25
Step 250/300: training loss=1.38, validation loss=1.07
Step 249/300: training loss=0.91
Step 248/300: training loss=0.87
Step 247/300: training loss=1.65
Step 246/300: training loss=0.65
Step 245/300: training loss=0.75
Step 244/300: training loss=1.35
Step 243/300: training loss=1.36
Step 242/300: training loss=1.21
Step 241/300: training loss=0.71
Step 240/300: training loss=0.51, validation loss=0.70
Step 239/300: training loss=1.00
Step 238/300: training loss=1.12
Step 237/300: training loss=0.94


# **Testing the new model**

In [84]:
response = client.fine_tuning.jobs.retrieve(job_id)
response

FineTuningJob(id='ftjob-pp0YXxTT9l9yTK55VrIUL4aD', created_at=1724247641, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:wiki-medical-terms:9ygDyXer', finished_at=1724249016, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-OAr1jvg8os1Ah0dcvn3JHKaZ', result_files=['file-9Pm5wud0q0waQFnXKGMGCmab'], seed=2097429054, status='succeeded', trained_tokens=1125156, training_file='file-5c4JoGvxeo9lDJ3ryzyMyNsz', validation_file='file-TC6MuUUKm4zZVfvMhgMA8dbq', estimated_finish=None, integrations=[], user_provided_suffix='wiki-medical-terms')

In [85]:
fine_tune_model_id = response.fine_tuned_model
fine_tune_model_id

'ft:gpt-3.5-turbo-0125:personal:wiki-medical-terms:9ygDyXer'

In [86]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})
test_message = "Hey what is Amblyopia?"

test_messages.append({"role": "user", "content": test_message})

print(test_messages)

[{'role': 'system', 'content': 'You are a chatbot designed to explain medical terms, please answer the patients questions like a doctor.'}, {'role': 'user', 'content': 'Hey what is Amblyopia?'}]


In [87]:
response = client.chat.completions.create(
    model = fine_tune_model_id,
    messages = test_messages
)

print(response.choices[0].message.content)

Amblyopia, often called lazy eye, is a condition that is caused by poor development of the visual system. It usually begins in infancy or childhood and it may get worse over time. The disorder is characterized by blurry vision in the affected eye as well as a difference in the clarity and sharpness of the image observed by each eye. With normal binocular vision, the brain is able to combine these two images into a single image. This ability to merge visual impressions received by both eyes is called fusion. But in children affected with amblyopia, the brain may learn to ignore or suppress the image from the affected eye. This can result in a loss of vision acuity in that eye and—a rare complication—a loss of depth perception. Amblyopia is the most common cause of decreased single-eye vision and the most common form of visual impairment in children.Neurological adaptations to the visual areas of the brain have also been detected in adults who wear strong correction lenses for refractive